# Loop 6 Analysis: Evaluating Stacking vs Alternatives

## Key Questions:
1. Should we submit exp_005 (stacking)?
2. What's the predicted LB for exp_005 based on CV-LB gap pattern?
3. Which alternative approach is most promising?

## Evaluator's Concerns:
- Stacking CV (0.81744) < CatBoost alone (0.81617-0.81951)
- Predicted rate 52.8% is concerning (higher than best LB submission 51.7%)
- CV-LB gap pattern suggests exp_005 will underperform

In [ ]:
import pandas as pd
import numpy as np

# CV-LB gap analysis
submissions = [
    {'exp': 'exp_000', 'cv': 0.80674, 'lb': 0.79705, 'pred_rate': None},
    {'exp': 'exp_003', 'cv': 0.81951, 'lb': 0.80453, 'pred_rate': 0.517},
    {'exp': 'exp_004', 'cv': 0.81928, 'lb': 0.80406, 'pred_rate': 0.538},
]

df = pd.DataFrame(submissions)
df['gap'] = df['cv'] - df['lb']
df['gap_pct'] = (df['gap'] / df['cv']) * 100
print("CV-LB Gap Analysis:")
print(df.to_string(index=False))
print(f"\nMean gap: {df['gap'].mean():.5f} ({df['gap_pct'].mean():.2f}%)")

In [ ]:
# Predict LB for exp_005
exp_005_cv = 0.81744
exp_005_pred_rate = 0.528

# Using average gap
mean_gap = df['gap'].mean()
predicted_lb_avg = exp_005_cv - mean_gap

# Using most recent gap (exp_004)
recent_gap = df[df['exp'] == 'exp_004']['gap'].values[0]
predicted_lb_recent = exp_005_cv - recent_gap

print(f"exp_005 CV: {exp_005_cv:.5f}")
print(f"exp_005 predicted rate: {exp_005_pred_rate:.3f}")
print(f"\nPredicted LB using average gap ({mean_gap:.5f}): {predicted_lb_avg:.5f}")
print(f"Predicted LB using recent gap ({recent_gap:.5f}): {predicted_lb_recent:.5f}")
print(f"\nBest LB so far (exp_003): 0.80453")
print(f"\nConclusion: exp_005 likely to score ~{predicted_lb_avg:.4f}-{predicted_lb_recent:.4f} on LB")
print(f"This would be {'WORSE' if predicted_lb_avg < 0.80453 else 'BETTER'} than exp_003")

In [ ]:
# Analyze prediction rate vs LB performance
print("Prediction Rate vs LB Performance:")
print("="*50)
print(f"Training transported rate: 50.4%")
print(f"\nexp_003: pred_rate=51.7% -> LB=0.80453 (BEST)")
print(f"exp_004: pred_rate=53.8% -> LB=0.80406 (worse)")
print(f"exp_005: pred_rate=52.8% -> LB=??? (predicted worse)")
print(f"\nPattern: Higher prediction rate = worse LB")
print(f"exp_003 is closest to training rate and has best LB")

In [ ]:
# What would improve LB?
print("\nStrategies to Improve LB:")
print("="*50)
print("\n1. WEIGHTED ENSEMBLE (Evaluator Option A)")
print("   - Weight CatBoost higher (0.6) since it's best single model")
print("   - May reduce prediction rate closer to training")
print("   - Quick to implement")

print("\n2. FEATURE SELECTION (Evaluator Option B)")
print("   - Remove bottom 20% features by importance")
print("   - Reduces overfitting directly")
print("   - 22 features have importance < 1.0")

print("\n3. REGULARIZED CATBOOST (Evaluator Option C)")
print("   - depth=6 (vs 8), l2_leaf_reg=5.0 (vs 3.52)")
print("   - subsample=0.8 for randomness")
print("   - Directly addresses overfitting")

print("\n4. CALIBRATION")
print("   - Adjust predictions to match training distribution")
print("   - Could help if prediction rate is the issue")

In [ ]:
# Decision matrix
print("\nDECISION MATRIX:")
print("="*50)
print("\nOption | Effort | Expected Impact | Risk")
print("-"*50)
print("Weighted Ensemble | Low | Medium | Low")
print("Feature Selection | Medium | High | Medium")
print("Regularized CatBoost | Low | Medium | Low")
print("Calibration | Low | Unknown | Medium")

print("\n\nRECOMMENDATION:")
print("="*50)
print("1. DO NOT submit exp_005 - predicted to underperform")
print("2. Try weighted ensemble first (quick win)")
print("3. Then try feature selection + regularization")
print("4. Submit only if CV improves AND pred_rate <= 51.7%")